# Extracting field experiment plot boundary using `plot-boundary-extractor` module

This is a tutorial on how to extract field experiment plot boundary using `plot-boundary-extractor` module. 

This repository is associated with our research on **Developing a segment anything model-based framework for automated plot extraction**, which has been submitted to *Precision Agriculture* and is currently under review.

---

*Written by [Hansae Kim](https://gdsl.org/team/hansae-kim/) and [Jinha Jung](https://gdsl.org/team/jinha-jung/)*

## `plot-boundary-extractor` installation

### Set up a virtual environment

First, we need to install `plot-boundary-extractor` on the machine you would like to run this example tutorial. We will set up a new Python Virtual Environment called `pbe`. 

```bash
$ conda create -n pbe -c conda-forge gdal python=3.10 -y
```

We need to activate the newly created virtual environment before proceeding.

```bash
$ conda activate pbe
```

### Install `plot-boundary-extractor` module

Now we need to install `plot-boundary-extractor` module using `pip`.

```bash
$ pip install plot-boundary-extractor
```

### Install torch

The `plot-boundary-extractor` module uses `torch` module, so we need to install it on the machine you would like to run this example tutorial. 

#### CPU version

If your machine is not equipped with NVIDIA GPU, then you will have to install the `torch` without GPU support.

```bash
$ pip install torch torchvision
```

#### GPU version

If your machine is equipped with NVIDIA GPU, then you will have to install the `torch` with GPU support.

```bash
$ pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
```

### Install `SAM`

The `plot-boundary-extractor` uses `SAM` to detect plot boundary automatically, so you have to install `SAM` using the below command.

```bash
$ pip install git+https://github.com/facebookresearch/segment-anything.git
```

You also have to download a pre-trained model using the below command. Please make a note where you're downloading the pre-trained model. We will need this path information in the later tutorial.

```bash
$ wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
```

Now the installation is done, and you're ready to go to the next step

## Importing modules

You have to load modules required for this tutorial

* `os` module: This module is required to work with file structure on the machine.
* `leafmap` module: This module is used to visualize orthomosaic images from D2S interactively and also specify the area of interests.
* `datetime` module: This module is used to specify the time windows to query orthomosaic images from D2S.
* `d2spy` module: This module is used to connect to D2S and stream required data products.
* `d2s_env` module: This is a custom module that contains my login information. A separate link will be added here later for more details.
* `plot_boundary_extractor` module: This is the one we will use to extract plot boundaries.

In [1]:
import os
import leafmap
from datetime import date
from d2spy.workspace import Workspace
import d2s_env
from plot_boundary_extractor import PlotExtraction

## Find the data product from D2S

This section will show you how to connect to D2S using `d2spy` module and find the data product to perform plot boundary extraction.

### Connect to a D2S workspace

You can use the code below to connect to a D2S instance. In this case, you are connecting to a PS2 (Plant Science 2.0 - https://www.purdue.edu/purduemoves/plant-sciences-2-0/) instance. The PS2 D2S instance is currently managed by Dr. Jinha Jung's research group (https://gdsl.org) and it is freely available as of March, 2025. 

D2S stores data using the following structure.

* Workspace
  * Project 1
    * Flight 1
      * Raw data
      * Data product 1
      * Data product 2
      * ...
      * Data product L
    * ...
    * Flight M
  * Project 2
  * ...
  * Project N
  
That being said, first step is to connect to a D2S workspace using the code below. 

In [3]:
# Connect to workspace
workspace = Workspace.connect("https://ps2.d2s.org")

# If you don't have d2s_env setup, then you will have to specify your login email. This will ask you type in your password.
# workspace = Workspace.connect("https://ps2.d2s.org", "YOUR@EMAIL.ADDRESS")

Not all data are publicly available in D2S, as the owner of the project has a full control on who can access the data. When the data are protected, you need to authenticate yourself to access those data programatically. For a programatic authentication, you need to grab your `API_KEY` generated from the D2S instance. You can do that using the code below. The below code is also setting a TiTiler URL for dynamic tiling. 

In [12]:
# Check for API key
api_key = workspace.api_key
if not api_key:
    print("No API key. Please request one from the D2S profile page and re-run this cell.")
else:
    os.environ["D2S_API_KEY"] = api_key
    
os.environ["TITILER_ENDPOINT"] = "https://tt.d2s.org"

### Find a project

Once you're connected to a D2S instance, you need to find a project that contains data products you need. In this example, we will use 2022 Cornell WheatCAP yield trial project as an example.

You can use `workspace.get_projects()` to get a list of all projects you have access, then filter the list further using `.filter_by_title()` function to narrow it down to a smaller list. 

In [6]:
# Change the search term in `.filter_by_title` to match your project
project = workspace.get_projects().filter_by_title("cornell")
for i, proj in enumerate(project):
    print(i, proj)

0 Project(title='2023 Cornell Wheat', description='USDA WheatCAP Project - Cornell University 2023 Winter Wheat Master Nursery trial at Ithaca, NY - McGowan', start_date=datetime.date(2022, 10, 11), end_date=datetime.date(2023, 6, 29))
1 Project(title='2024 Cornell Wheat', description='USDA WheatCAP Project - Cornell University 2024 Winter Wheat Master Nursery trial at Ithaca, NY - Helfer', start_date=datetime.date(2023, 10, 11), end_date=datetime.date(2024, 7, 8))
2 Project(title='2022 Cornell Wheat', description='USDA WheatCAP Project - Cornell University 2022 WWMASTER2022ACCT3 trial at Ithaca, NY - Helfer', start_date=datetime.date(2021, 10, 21))


The above code resulted returned 3 D2S projects, and we will choose the 2022 trial.

In [7]:
project = project[2]

### Find a flight

Once you choose a project, the selected project may have multiple flights. You can use `project.get_flights()` function to get a list of all flights in the selected project, then filter the list further using `.filter_by_date()` function to narrow it down to a smaller list.

In [8]:
# Change the date range in `filter_by_date` to match the acquistion date of the flight in your project
start_date = date(2022,4,1)
end_date = date(2022,5,31)
flights = project.get_flights().filter_by_date(start_date,end_date)
for i, flight in enumerate(flights):
    print(i,flight)

0 Flight(acquisition_date='2022-04-20', name='', altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='Multispectral', platform='M300')
1 Flight(acquisition_date='2022-05-11', name=None, altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='RGB', platform='Phantom_4')
2 Flight(acquisition_date='2022-05-25', name='', altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='Multispectral', platform='Phantom_4')
3 Flight(acquisition_date='2022-04-20', name=None, altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='RGB', platform='Phantom_4')
4 Flight(acquisition_date='2022-04-30', name=None, altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='RGB', platform='Phantom_4')
5 Flight(acquisition_date='2022-04-30', name='', altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='Multispectral', platform='M300')
6 Flight(acquisition_date='2022-05-11', name='', altitude=120.0, side_overlap=60.0, forward_overlap=75.0, sensor='Multispe

After filtering flights by a specific time window, we got 6 flights in the list. We will use a RGB flight collected on `05/11/2022` in this tutorial

In [9]:
flight = flights[1]

### Find a data product

Each flight can contain multiple data products. Let's check what data products are available from the selected flight first.

In [10]:
data_products = flight.get_data_products()

for i, data_product in enumerate(data_products):
    print(i, data_product)

0 DataProduct(data_type='ortho', filepath='/static/projects/b4ab960b-9629-46d7-881c-5612fd5ee0dd/flights/0d73f050-6a66-4922-851c-9b98e2a45dab/data_products/58822a09-c290-41ac-8248-a306fe5299cf/7c782101-90a6-42a5-b39f-c5b70d0cfb4a.tif', original_filename='20220511_cn_mic_dry_mosaic_rgb.tif', is_active=True, public=True, stac_properties={'raster': [{'data_type': 'uint8', 'stats': {'minimum': 0.0, 'maximum': 255.0, 'mean': 112.381, 'stddev': 89.018}}, {'data_type': 'uint8', 'stats': {'minimum': 0.0, 'maximum': 255.0, 'mean': 115.057, 'stddev': 66.764}}, {'data_type': 'uint8', 'stats': {'minimum': 0.0, 'maximum': 255.0, 'mean': 73.811, 'stddev': 57.792}}, {'data_type': 'uint8', 'stats': {'minimum': 0.0, 'maximum': 255.0, 'mean': 93.907, 'stddev': 122.995}}], 'eo': [{'name': 'b1', 'description': 'Red'}, {'name': 'b2', 'description': 'Green'}, {'name': 'b3', 'description': 'Blue'}, {'name': 'b4', 'description': 'Alpha'}]}, status='SUCCESS', url='https://ps2.d2s.org/static/projects/b4ab960b-9

As you can see from the above, there are 4 data products available in the selected flight. We need an orthomosaic image for plot boundary extraction, so we need to choose `ortho` data product.

In [11]:
data_product = data_products[0]

### Visualize the data product

Now, we can visualize the selected data product using `leafmap` module using the code below.

In [ ]:
m = leafmap.Map()
m.clear_layers()
m.add_basemap("USGS NAIP Imagery")
m.add_cog_layer(f"{data_product.url}?API_KEY={api_key}", name="ortho", zoom_to_layer=True)
m

## Extracting plot boundaries

Now, we can see an orthomosaic image in an interactive map. You can zoom into a area where your field experiment is located. We need the following information to run the `plot-boundary-extractor` code.

* `base_layer`: This is the data product to run the algorithm on. The selected data product is used here automatically in the code below.
* `api_key`: This is the `API_KEY` to access the selected data product. 
* `clipped_filename`: where to save the clipped orthomosaic image
* `clip_boundary`: This is a boundary of your field experiment. You can use the drawing tool (located in the left side of the map) to draw a polygon around your field experiment. The last drawn polygon will be used here.
* `n_rows`: Number of rows in the field experiment
* `n_cols`: Number of columns in the field experiment
* `plot_width`: Width of each plot (in the unit of the data product, i.e., if the data product is in meter, then it will be in meter)
* `plot_height`: Height of each plot (in the unit of the data product, i.e., if the data product is in meter, then it will be in meter)
* `resize`: (Optional) You can optionally set (height, width) to make the clipped image smaller. This can be useful when you don't have a powerful GPU to run this code. When specifying the image size, keep the image aspect ratio as much as possible.
* `points_per_side`: (Optional) Number of seed points to use on each side
* `iou_threshold`: (Optional) IoU threshold value to determine initial plot candidates
* `cc_coverage_thr`: (Optional) Canopy Cover threshold value to determine initial plot candidates
* `out_filename`: Output file name for automatically detected plot boundaries in GeoJSON format
* `sam_checkpoint`: Full path to the `SAM` pre-trained model

Once you specify the above in `arg` dictionary, now you can perform the plot boundary extraction using the code below.


In [ ]:
# define arguments
args = {
        'base_layer': data_product, 
        'api_key': api_key,
        'clipped_filename': './2022_cornell_wheat_example.tif', 
        'clip_boundary': m.draw_control.last_draw, #optional
        'n_rows':22,
        'n_cols': 13,
        'plot_width': 3.7,
        'plot_height': 1.0,
        # 'resize': (320, 640), # optional
        'points_per_side': 10, # optional 
        'iou_threshold': 0.1, # optional
        'cc_coverage_thr': 0, # optional
        'out_filename': './2022_cornell_wheat_plot_boundary.geojson', # path to save plot boundary
        'sam_checkpoint': "./sam_vit_h_4b8939.pth" # path to SAM checkpoint
        }

# automatic detection
plot = PlotExtraction(**args)
plot.automatic_detection(save=True)
plot.show(m)

Loaded image: ./2022_cornell_wheat_example.tif
Loaded SAM automatic maskgenerator: points per side=32, device=cuda
Resized image: (1060, 2121)
Estimated orientation angle: 25.02 degree
Loaded SAM automatic maskgenerator: points per side=10, device=cuda
Initial plots: 74
Loaded SAM predictor
Detected missing plots: 212
Refined plots: 283
Assigned rows and columns
Process completed


Map(bottom=99198733.0, center=[42.44629208201381, -76.43943920731544], controls=(ZoomControl(options=['positio…

## Manual plot boundary operation

Although the above procedure works well in most cases, there could be some cases where you may to manually delete some plots or add any missing plots. Following instruction shows you how to do manual plot boundary deletion or addition. 

### Remove a plot

You can use the `leafmap` built-in function to identify an `id` of the plot you would like to delete. You can delete a specific plot using the code below. 

In [ ]:
id = [122]
plot.delete(id)
plot.show(m)

### Add plot

When there is any plot missing from the generated boundaries, you can using the drawing tool to draw a small polygon within the missing plot, then run the following code to manually add the missing plot. 

In [ ]:
plot.add(m.draw_control.last_draw)
plot.show(m)

## Export the plot boundary to GeoJSON file

When everything is done, now you can export the extracted plot boundary to a final GeoJSON file. Since the GeoJSON format standard requires `EPSG:4326`, the below code will transform the data into `EPSG:4326` and save it to your machine.

In [ ]:
plot.to_file('plot_boundary.geojson')

# Step by Step Example

In [ ]:
# define arguments
args = {
        'base_layer': data_product, 
        'api_key': api_key,
        'clipped_filename': in_filename,
        'clip_boundary': m.draw_control.last_draw, #optional
        'n_rows': n_rows,
        'n_cols': n_cols,
        'plot_width': plot_width,
        'plot_height': plot_height,
        'resize': (1024,1024), # optional
        'points_per_side': 64, # optional 
        'iou_threshold': 0.1, # optional
        'cc_coverage_thr': 0, # optional
        'out_filename': 'plot_boundary.geojson',
        'sam_checkpoint': "/data/hans/segment-anything/sam_vit_h_4b8939.pth" # manual download
        }

plot = PlotExtraction(**args)

# Visualize the base layer

In [ ]:
m = leafmap.Map()
m.clear_layers()
m.add_basemap("USGS NAIP Imagery")
m.add_cog_layer(f"{data_product.url}?API_KEY={api_key}", name="ortho", zoom_to_layer=True)
m

# Load image and rotate if needed

In [ ]:
# load image to the plot object
plot.load_image()

# load sam model and get initial plots
processing_time = 0
sam_checkpoint = "./sam_vit_h_4b8939.pth"
plot.load_sam(sam_checkpoint, points_per_side=16)
masks = plot.get_masks()

# rotate plot if needed
img_rotated = plot.rotate_plot()

# visualize the results
# from skimage.color import label2rgb

# plt.figure(figsize=(5, 15))
# plt.imshow(plot.img_array)
# plt.imshow(label2rgb(masks), alpha=0.4)
# plt.xticks([])
# plt.yticks([])
# plt.show()

# plt.imshow(img_rotated)
# plt.xticks([])
# plt.yticks([])
# plt.show()

# Get initial plots

In [ ]:
plot.load_sam(sam_checkpoint, points_per_side=100)
initial_plots = plot.initial_plots()

initial_plots.set_crs(f'EPSG:{plot.epsg}', inplace=True)
gdf_geojson = plot.to_geojson(initial_plots, rotation=True)
m.add_geojson(gdf_geojson, layer_name="Initial plot boundary", 
              style={"color": "red", "weight": 1, "fill": False}, zoom_to_layer=True)
m

# Grid filling

In [ ]:
plot.load_sam(plot.sam_checkpoint, type='manual')
gdf_filled = plot.grid_filling()
print(f"Processing time: {processing_time:.2f} seconds")
gdf_filled.set_crs(f'EPSG:{plot.epsg}', inplace=True)
gdf_geojson = plot.to_geojson(gdf_filled, rotation=True)

m.add_geojson(gdf_geojson, layer_name="Filled plot boundary", 
              style={"color": "yellow", "weight": 1, "fill": False}, zoom_to_layer=True)
m

# Grid remove

In [ ]:
gdf_removed = plot.grid_remove(gdf_filled)
gdf_final = plot.assign_row_col(gdf_removed)
gdf_final.set_crs(f'EPSG:{plot.epsg}', inplace=True)
gdf_geojson = plot.to_geojson(gdf_final, rotation=True)

In [ ]:
gdf_geojson = plot.to_geojson(gdf_final, rotation=True)
m.add_geojson(gdf_geojson, layer_name="Refined plot boundary", 
              style={"color": "cyan", "weight": 2, "fill": False}, zoom_to_layer=True)
m

# Export to GeoJSON file

In [ ]:
gdf_final.to_file('plot_boundary.geojson', driver='GeoJSON')